## Sample Model

Based on the exploration, we try the following *as a starting point*.

> #### Delete the following columns
> - days_with_fog
> - direction_peak_wind_speed
> - direction_max_wind_speed
> - max_wind_speed
> - year_built
> - days_above_90F
> - days_above_110F
> - facility_type
> - site_eui (obviously, because target column)
> - For now also delete Year_Factor

> #### Impute missing values for energy_star_rating 
> Imputing is done by replacing nan by the mean <br>
> By second thoughts we do NOT impute as XGBoost infers missing values

> #### One-hot encode categorical values 
> - State_Factor
> - Building_Class

For the sample model, we use **random forest** and **xgboost**.

In [36]:
import sklearn
import xgboost as xgb
from evaluation import RMSE
from evaluation import helper_func
import pandas as pd
import gc
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
import lightgbm as lgbm
import time
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import KFold
import optuna
import math

#### Preparing and reading data

In [2]:
data = pd.read_csv("/Users/charlottefelius/documents/wids2022/WIDS/data/train.csv")
submission = pd.read_csv("/Users/charlottefelius/documents/wids2022/WIDS/data/test.csv")

In [3]:
len(submission)

9705

### Feature engineering

In [4]:
len(data.facility_type.unique())

60

#### delete columns

In [5]:
reduced = pd.read_csv("/Users/charlottefelius/documents/wids2022/WIDS/data/train.csv")

to_delete = ["days_with_fog", "direction_peak_wind_speed", "direction_max_wind_speed", "max_wind_speed", "year_built", 
             "days_above_90F", "days_above_110F", "facility_type", "Year_Factor"]

# to_delete_less = ["days_with_fog", "direction_peak_wind_speed", "direction_max_wind_speed", "max_wind_speed"]
to_delete_less = ["days_with_fog", "direction_peak_wind_speed", "max_wind_speed"]

def delete_cols(dataframe, columns):
    for colname in columns:
        del dataframe[colname]

delete_cols(reduced, to_delete_less)
delete_cols(submission, to_delete_less)

# # collect garbage
# gc.collect()


#### Impute variables 

In [6]:
# impute with mean, ensures highest correlation

reduced['energy_star_rating'] = reduced['energy_star_rating'].fillna(56.0)
submission['energy_star_rating'] = submission['energy_star_rating'].fillna(56.0)
reduced['year_built'] = reduced['year_built'].fillna(reduced['year_built'].median())
submission['year_built'] = submission['year_built'].fillna(submission['year_built'].median())

In [7]:
fillmedian = ["direction_max_wind_speed"]

for i in fillmedian:
    reduced[i] = reduced[i].fillna(reduced[i].median())
    submission[i] = submission[i].fillna(submission[i].median())

In [8]:
# test which value enhances highest correlation still
# highest_corr = []

# for i in data['energy_star_rating'].unique():
#     d = pd.read_csv("/Users/charlottefelius/documents/wids2022/WIDS/data/train.csv")
#     d['energy_star_rating'] = d['energy_star_rating'].fillna(i)
#     correlations = helper_func.get_correlation(d, 'site_eui', 0)[1]
#     highest_corr.append((i, correlations[1]))

#### Create new columns

In [9]:
# below compute sum of below 30 and sum of above 80
days_ = [string for string in reduced.columns if 'days_' in string]

In [10]:
# # below compute sum of below 30 and sum of above 80
# reduced['d_below_30_F'] = reduced[list(reduced.filter(regex='days_below_'))].sum(axis=1)
# reduced['d_above_80_F'] = reduced[list(reduced.filter(regex='days_above_'))].sum(axis=1)
# submission['d_below_30_F'] = submission[list(submission.filter(regex='days_below_'))].sum(axis=1)
# submission['d_above_80_F'] = submission[list(submission.filter(regex='days_above_'))].sum(axis=1)

In [11]:
# delete the other columns
# delete_cols(reduced, days_)
# delete_cols(submission, days_)

#### Onehot encoding

In [12]:
onehot = ["facility_type", "State_Factor", "building_class"]

def onehotter(dataframe, to_onehot):
    
    for i in to_onehot:
        ohe_df = pd.get_dummies(dataframe[i], prefix=i)

        # concat with original data
        dataframe = pd.concat([dataframe, ohe_df], axis=1).drop([i], axis=1)
        
    return dataframe

In [13]:
submission = onehotter(submission, onehot)
reduced = onehotter(reduced, onehot)

In [14]:
# save and pop submission id apart
Y_test_sub = submission["id"]
submission.pop("id")

0       75757
1       75758
2       75759
3       75760
4       75761
        ...  
9700    85457
9701    85458
9702    85459
9703    85460
9704    85461
Name: id, Length: 9705, dtype: int64

#### Evaluation

In [48]:
def RMSE(original, predicted):
    
    aggregate = 0
    
    for orig, pred in zip(original, predicted):
        aggregate+= (orig - pred)**2
    
    RMSE_ = math.sqrt(1/len(original) * aggregate)
        
    print(f'RMSE: {RMSE_}')
    
    return RMSE

## Sample model

In [15]:
path = "/Users/charlottefelius/documents/WIDS2022/WIDS/data/train.csv"

#### Split into train and test set about  30 / 70 ratio

In [16]:
X = reduced
y = X['site_eui']
y_id = X['id']
X.pop('site_eui')
X.pop('id')
X.pop('State_Factor_State_6')

0        0
1        0
2        0
3        0
4        0
        ..
75752    0
75753    0
75754    0
75755    0
75756    0
Name: State_Factor_State_6, Length: 75757, dtype: uint8

In [183]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=123)

In [18]:
raise NotImplementedError()

NotImplementedError: 

### XGboost

In [ ]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [ ]:
model = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.2,
                max_depth = 100, alpha = 0.2, n_estimators = 40)

In [ ]:
# Highest score RMSE: 38.672019

model.fit(X_train, y_train)
preds = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

In [ ]:
raise NotImplementedError()

In [ ]:
MODEL_MAX_DEPTH = 12
MODEL_TASK_TYPE = 'GPU'
MODEL_RL = 0.025
MODEL_EVAL_METRIC ='RMSE'
MODEL_LOSS_FUNCTION = 'RMSE'
MODEL_ESR = 10
MODEL_VERBOSE = 1000
MODEL_ITERATIONS = 28000

### LightGBM

In [225]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

In [230]:
model = lgbm.LGBMRegressor(max_depth=10, learning_rate=0.05, n_estimators=5000)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
# 37.48368551936998

predicted = model.predict(X_test)

In [ ]:
RMSE(y_test, predicted)

#### Evaluation

In [ ]:
# N_SPLITS = 20
# strat_kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=1121218)

# scores = np.empty(N_SPLITS)
# for idx, (train_idx, test_idx) in enumerate(strat_kf.split(X, y)):
#     print("=" * 12 + f"Training fold {idx}" + 12 * "=")
#     start = time.time()

#     X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
#     y_train, y_val = y[train_idx], y[test_idx]
#     eval_set = [(X_val, y_val)]

#     model = lgbm.LGBMRegressor()
#     model.fit(
#         X_train,
#         y_train,
#         eval_set=eval_set,
#         categorical_feature=cat_idx,
#         early_stopping_rounds=200,
#         eval_metric="binary_logloss",
#         verbose=False,
#     )

#     preds = model.predict(X_val)
#     loss = log_loss(y_val, preds)
#     scores[idx] = loss
#     runtime = time.time() - start
#     print(f"Fold {idx} finished with score: {RMSE:.5f} in {runtime:.2f} seconds.\n")

In [23]:
rkf = RepeatedKFold(n_splits=3, n_repeats=3, random_state=42)

params = {
        "objective": "binary",
        "metric": "binary_error",
        "verbosity": -1,
        "boosting_type": "gbdt",                
        "seed": 42
    }

study_tuner = optuna.create_study(direction='minimize')
dtrain = lgbm.Dataset(X, label=y)

# Suppress information only outputs - otherwise optuna is 
# quite verbose, which can be nice, but takes up a lot of space
# optuna.logging.set_verbosity(optuna.logging.WARNING) 

# # Run optuna LightGBMTunerCV tuning of LightGBM with cross-validation
# tuner = lgbm.LightGBMTunerCV(params, 
#                             dtrain, 
#                             study=study_tuner,
#                             verbose_eval=False,                            
#                             early_stopping_rounds=250,
#                             time_budget=19800, # Time budget of 5 hours, we will not really need it
#                             seed = 42,
#                             folds=rkf,
#                             num_boost_round=10000,
#                             callbacks=[lgbm.reset_parameter(learning_rate = [0.005]*200 + [0.001]*9800) ] #[0.1]*5 + [0.05]*15 + [0.01]*45 + 
#                            )

# tuner.run()

In [ ]:
print(tuner.best_params)
# Classification error
print(tuner.best_score)
# Or expressed as accuracy
print(1.0-tuner.best_score)

In [29]:
num_folds = 5
kf = KFold(n_splits = num_folds, random_state = 42, shuffle=True)
error = 0
models = []
for i, (train_index, val_index) in enumerate(kf.split(X)):
    if i + 1 < num_folds:
        continue
    print(train_index.max(), val_index.min())
    train_X = X[train_index]
    val_X = X[val_index]
    train_y = y[train_index]
    val_y = y[val_index]
    lgb_train = lgb.Dataset(train_X, train_y > 0)
    lgb_eval = lgb.Dataset(val_X, val_y > 0)
    params = {
            'boosting_type': 'gbdt',
            'objective': 'regression',
                    'metric': {'rmse'},
            'learning_rate': 0.1,
            'feature_fraction': 0.8,
            'bagging_fraction': 1.0,
            'bagging_freq' : 0
             
            }
    gbm_class = lgb.train(params,
                lgb_train,
                num_boost_round=2000,
                valid_sets=(lgb_train, lgb_eval),
               early_stopping_rounds=20,
               verbose_eval = 20)
    
    lgb_train = lgb.Dataset(train_X[train_y > 0], train_y[train_y > 0])
    lgb_eval = lgb.Dataset(val_X[val_y > 0] , val_y[val_y > 0])
    params = {
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': {'rmse'},
            'learning_rate': 0.1,
            'feature_fraction': 0.8,
            'bagging_fraction': 1.0,
            'bagging_freq' : 0
                                }
    gbm_regress = lgb.train(params,
                lgb_train,
                num_boost_round=2000,
                valid_sets=(lgb_train, lgb_eval),
               early_stopping_rounds=20,
               verbose_eval = 20)
#     models.append(gbm)

    y_pred = (gbm_class.predict(val_X, num_iteration=gbm_class.best_iteration) > .5) *\
    (gbm_regress.predict(val_X, num_iteration=gbm_regress.best_iteration))
    error += np.sqrt(mean_squared_error(y_pred, (val_y)))/num_folds
    print(np.sqrt(mean_squared_error(y_pred, (val_y))))
    break
print(error)

75756 2


KeyError: "None of [Int64Index([    0,     1,     3,     4,     6,     7,     8,     9,    10,\n               11,\n            ...\n            75744, 75746, 75747, 75748, 75749, 75751, 75752, 75754, 75755,\n            75756],\n           dtype='int64', length=60606)] are in the [columns]"

In [ ]:
raise NotImplementedError()

### Catboost (Does not work on Apple M1)

In [ ]:
MODEL_MAX_DEPTH = 12
MODEL_TASK_TYPE = 'GPU'
MODEL_RL = 0.025
MODEL_EVAL_METRIC ='RMSE'
MODEL_LOSS_FUNCTION = 'RMSE'
MODEL_ESR = 10
MODEL_VERBOSE = 1000
MODEL_ITERATIONS = 28000

In [ ]:
model = CatBoostRegressor(
    verbose=1000,
    early_stopping_rounds=10,
    #random_state=41,
    random_seed=535,
    max_depth=MODEL_MAX_DEPTH,
    task_type=MODEL_TASK_TYPE,
    learning_rate=MODEL_RL,
    iterations=28000
    
)


In [ ]:
# Highest score RMSE: 38.672019

model.fit(X_train, y_train)
preds = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

In [ ]:
raise NotImplementedError()

#### Fit testset, write to .csv

In [175]:
# Now fit on whole training set
model.fit(X, y)
predicted = model.predict(submission)

In [177]:
# Convert Preds to DataFrame 

result = pd.DataFrame(Y_test_sub)
result["site_eui"] = predicted

In [178]:
# Write to CSV file

result.to_csv('/Users/charlottefelius/documents/WIDS2022/WIDS/results/attempt11.csv', index=False, header=True)